In [1]:
# -------------------------------------------------------------
# 작성자 : 백강민
# 작성목적 : SKALA Python Day1 - generator 실습 코드 2
# 작성일 : 2025-01-12
# 변경사항 내역 :
#   2025-01-12 - 최초 작성
# -------------------------------------------------------------

import sys
import time


def even_square_gen(n: int):
    """0 이상 n 미만 정수 중 '짝수만' 제곱해서 하나씩 생성 (yield)"""
    for i in range(0, n, 2):
        yield i * i


def format_memory(size_bytes: int) -> str:
    kb = size_bytes / 1024
    mb = kb / 1024
    return f"{size_bytes:,} bytes ({kb:.2f} KB, {mb:.2f} MB)"


def format_time(seconds: float) -> str:
    return f"{seconds:.6f} sec ({seconds * 1000:.3f} ms)"


def speed_summary(list_time: float, gen_time: float) -> tuple[str, str]:
    """측정 결과 기반 속도 요약 (빠름/느림 + 배수)"""
    if list_time <= 0 or gen_time <= 0:
        return "측정값 0 sec", "측정값 0 sec"

    if list_time < gen_time:
        ratio = gen_time / list_time
        return f"더 빠름 (약 {ratio:.2f}x)", f"더 느림 (약 {ratio:.2f}x)"
    if gen_time < list_time:
        ratio = list_time / gen_time
        return f"더 느림 (약 {ratio:.2f}x)", f"더 빠름 (약 {ratio:.2f}x)"
    return "동일", "동일"


N = 1_000_000  # 0 ~ 999,999

# =============================================================
# 1) 리스트 방식: 짝수 제곱들의 총합 (리스트만 사용)
# =============================================================
print("1. 리스트 방식: 짝수 제곱들의 총합 계산")

t1_start = time.perf_counter()
sq_list = [i * i for i in range(0, N, 2)]  # 리스트 생성
list_sum = sum(sq_list)                   # 총합
t1_end = time.perf_counter()

list_time = t1_end - t1_start
list_mem = sys.getsizeof(sq_list)

print(f"   - 총합(리스트): {list_sum}\n")


# =============================================================
# 2) 제너레이터 방식: even_square_gen(n)으로 총합
# =============================================================
print("2. 제너레이터 방식: even_square_gen(n)으로 짝수 제곱 총합 계산")

t2_start = time.perf_counter()
gen_sum = sum(even_square_gen(N))  # 제너레이터로 총합
t2_end = time.perf_counter()

gen_time = t2_end - t2_start
gen_mem = sys.getsizeof(even_square_gen(N))  # 제너레이터 객체 자체 크기

print(f"   - 총합(제너레이터): {gen_sum}\n")


# =============================================================
# 3) 1,2번 결과 기반: 메모리/속도 비교
# =============================================================
print("3. 두 방법의 메모리 사용 차이 및 처리 속도 비교")

# 결과 검증 (같아야 정상)
if list_sum != gen_sum:
    print("   [경고] 총합이 다릅니다. 로직을 다시 확인하세요.\n")

time_diff = list_time - gen_time
mem_diff = list_mem - gen_mem
list_speed_desc, gen_speed_desc = speed_summary(list_time, gen_time)

print(f"   - 처리 시간(리스트): {format_time(list_time)} / {list_speed_desc}")
print(f"   - 처리 시간(제너레이터): {format_time(gen_time)} / {gen_speed_desc}")
print(f"   - 처리 시간 차이(리스트 - 제너레이터): {format_time(time_diff)}\n")

print(f"   - 리스트 객체 메모리: {format_memory(list_mem)}")
print(f"   - 제너레이터 객체 메모리: {format_memory(gen_mem)}")
print(f"   - 메모리 차이(리스트 - 제너레이터): {format_memory(mem_diff)}\n")

# =============================================================
# 실습 정리 포인트 (측정 결과 기반)
# =============================================================
print("【 실습 정리 포인트 】")
print(f"{'구분':<16} | {'일반 리스트 방식':<40} | {'제너레이터 방식':<40}")
print("-" * 105)

print(
    f"{'메모리 사용량':<16} | "
    f"{format_memory(list_mem):<40} | "
    f"{format_memory(gen_mem):<40}"
)
print(
    f"{'처리 방식':<16} | "
    f"{'모든 요소를 한 번에 메모리에 적재':<40} | "
    f"{'한 개씩 순차 처리 (Lazy)':<40}"
)
print(
    f"{'속도':<16} | "
    f"{format_time(list_time)} / {list_speed_desc:<16} | "
    f"{format_time(gen_time)} / {gen_speed_desc:<16}"
)


1. 리스트 방식: 짝수 제곱들의 총합 계산
   - 총합(리스트): 166666166667000000

2. 제너레이터 방식: even_square_gen(n)으로 짝수 제곱 총합 계산
   - 총합(제너레이터): 166666166667000000

3. 두 방법의 메모리 사용 차이 및 처리 속도 비교
   - 처리 시간(리스트): 0.016245 sec (16.245 ms) / 더 빠름 (약 1.65x)
   - 처리 시간(제너레이터): 0.026797 sec (26.797 ms) / 더 느림 (약 1.65x)
   - 처리 시간 차이(리스트 - 제너레이터): -0.010551 sec (-10.551 ms)

   - 리스트 객체 메모리: 4,167,352 bytes (4069.68 KB, 3.97 MB)
   - 제너레이터 객체 메모리: 224 bytes (0.22 KB, 0.00 MB)
   - 메모리 차이(리스트 - 제너레이터): 4,167,128 bytes (4069.46 KB, 3.97 MB)

【 실습 정리 포인트 】
구분               | 일반 리스트 방식                                | 제너레이터 방식                                
---------------------------------------------------------------------------------------------------------
메모리 사용량          | 4,167,352 bytes (4069.68 KB, 3.97 MB)    | 224 bytes (0.22 KB, 0.00 MB)            
처리 방식            | 모든 요소를 한 번에 메모리에 적재                      | 한 개씩 순차 처리 (Lazy)                       
속도               | 0.016245 sec (16.245 ms) / 더 빠름 (약 1.